## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
PATH = "data/cifar10/"
os.makedirs(PATH,exist_ok=True)

In [3]:
torch.cuda.set_device(1)

In [4]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [62]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, pad=sz//8, 
        aug_tfms=[RandomFlip()])#, RandomLighting(0.1,0.1), RandomRotate(2)])
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [63]:
bs=256

In [64]:
data = get_data(32,bs)

In [65]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=1):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2)
        self.bn = nn.BatchNorm2d(nf, momentum=0.01)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        
    def forward(self, x): return self.relu(self.bn(self.conv(x)))

class ResLayer(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.conv1=ConvLayer(ni, ni//2, ks=1)
        self.conv2=ConvLayer(ni//2, ni, ks=3)
        
    def forward(self, x): return self.conv2(self.conv1(x)) + x

In [66]:
class Darknet(nn.Module):
    def make_group_layer(self, ch_in, num_blocks, stride=1):
        return [ConvLayer(ch_in,ch_in*2,stride=stride)
               ] + [(ResLayer(ch_in*2)) for i in range(num_blocks)]

    def __init__(self, num_blocks, num_classes, start_nf=32):
        super().__init__()
        nf = start_nf
        layers = [ConvLayer(3, nf, ks=3, stride=1)]
        for i,nb in enumerate(num_blocks):
            layers += self.make_group_layer(nf, nb, stride=2-(i==1) )
            nf *= 2
        layers += [nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(nf, num_classes)]
        self.layers = nn.Sequential(*layers)

    def forward(self, x): return self.layers(x)

In [67]:
lr = 1.3

In [70]:
m = Darknet([2, 2, 2, 2], num_classes=10, start_nf=64)
learn = ConvLearner.from_model_data(m, data)

learn.crit = nn.CrossEntropyLoss()
wd=1e-4

In [71]:
# darknet 2222 lr 1.3 bn_mom 0.01
%time learn.fit(lr, 1, wds=wd, cycle_len=60, use_clr_beta=(30, 20, 0.95, 0.85))

A Jupyter Widget

 14%|█▍        | 27/196 [00:08<00:53,  3.17it/s, loss=2.17] 
epoch      trn_loss   val_loss   accuracy                   
    0      1.607285   1.782385   0.3763    
    1      1.272363   1.636895   0.4246                     
    2      1.033237   1.758246   0.4948                     
    3      0.896742   1.085012   0.6058                      
    4      0.776542   1.025511   0.6519                      
    5      0.705557   0.857516   0.6926                      
    6      0.641807   0.85887    0.7079                      
    7      0.593627   1.147322   0.6337                      
    8      0.552531   0.788334   0.7201                      
    9      0.529694   1.137728   0.6404                      
    10     0.537546   1.072297   0.6236                      
    11     0.508255   0.760872   0.7538                      
    12     0.492089   1.344845   0.638                       
    13     0.468839   0.802699   0.7318                      
    14     0.467246   0.838831

[array([0.21802]), 0.9405]

In [51]:
# darknet 2222 lr 1.3 65 cl
%time learn.fit(lr, 1, wds=wd, cycle_len=65, use_clr_beta=(30, 20, 0.95, 0.85))

A Jupyter Widget

epoch      trn_loss   val_loss   accuracy                   
    0      1.533084   1.725181   0.4627    
    1      1.224625   1.241789   0.5727                     
    2      0.995259   1.005162   0.6476                      
    3      0.865179   0.949657   0.6642                      
    4      0.756122   0.854677   0.6968                      
    5      0.692097   1.110497   0.6578                      
    6      0.635014   0.805524   0.7227                      
    7      0.588518   0.759759   0.7334                      
    8      0.567764   0.868924   0.7131                      
    9      0.547826   0.700656   0.7645                      
    10     0.524676   1.005113   0.6889                      
    11     0.50653    0.721323   0.7645                      
    12     0.493718   1.125408   0.6608                      
    13     0.479434   0.756994   0.7639                      
    14     0.475674   0.73913    0.7589                      
    15     0.464452   0.6123

[array([0.23373]), 0.9422]